## 1. Embed Data Manually Using InterSystems IRIS SQL
In this section, you will use InterSystems IRIS as vector storage for a data set that contains 1,000 synthetic patient case reports. While not in a codified clinical form, these free text reports about patients can serve as a good sample use case for a generative AI application.

You will begin by loading and viewing this data, and then you will generate vector embeddings for each case report. By generating embeddings, you will be able to run some simple vector searches to return relevant information based on a search string.

Let's begin by running the block of code below, which imports the necessary components to get started. This includes the *sentence_transformers* library that will be used to generate embeddings for this data.

In [ ]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
load_dotenv(override=True)

Next, we will set InterSystems IRIS-specific information such as username, password, the hostname and port of the InterSystems IRIS container in this lab, the namespace, and a connection string putting all of those elements together.

In [7]:
username = '_SYSTEM'
password = 'sys'
hostname = 'localhost'
port = 1972
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

Using the the connection string we just built, let's establish a connection to InterSystems IRIS.

In [8]:
engine = create_engine(CONNECTION_STRING)

### Load data set with 1,000 patient case reports
Next, we will load the JSON file with 1,000 case reports into a Pandas DataFrame that can be easily imported into InterSystems IRIS as a SQL table. A Pandas DataFrame is a powerful data structure that allows for efficient data manipulation and analysis. It provides a convenient way to handle and preprocess data, making it easy to clean, transform, and organize the data into a structured format suitable for SQL operations.

By using a DataFrame, we can leverage Pandas' robust functionality to ensure the data is correctly formatted and ready for seamless integration into InterSystems IRIS.

In [ ]:
import pandas as pd

# Load JSONL file into DataFrame
file_path = './data/healthcare/augmented_notes_1000.jsonl'
df_cases = pd.read_json(file_path, lines=True)
pd.set_option('display.max_rows', 1000)

Let's display the entire set of case reports to get a comprehensive view of the data by running the line below. This will help you understand the structure and content of the dataset before we proceed. Scroll through the data and see the types of case reports that exist, noting some of the patients and their situations referenced in the reports.

In [ ]:
df_cases

Using its vector storage capability, InterSystems IRIS supports vectors as a datatype in tables. In the next block, we will create a new table in InterSystems IRIS for our data to be loaded into: the *case_reports_sql* table. This table has columns for *note* and *note_vector*. The *note_vector* column will be used to store a vector embedding for each case report in the data set.

In [9]:
## DROP TABLE FIRST, IF ALREADY EXISTS .. remove this from the lab before finalizing
with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                DROP TABLE IF EXISTS case_reports_sql
        """
        result = conn.execute(text(sql))

In [10]:

with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE case_reports_sql (
        note VARCHAR(2500),
        note_vector VECTOR(DOUBLE, 384)
        )
                """
        result = conn.execute(text(sql))

### Create vector embeddings using a sentence transformer
Before we load the case reports into this *case_reports_sql* table in InterSystems IRIS, we will first create the vector embeddings that go with each report. Vector embeddings are numerical representations of text that capture the semantic meaning of the text, making it easier to perform tasks like similarity search, clustering, and classification. To generate these embeddings, we will use a pre-trained sentence transformer model.

Sentence transformers are a type of model designed to create dense vector representations of sentences, which can be used for various natural language processing tasks. These models are trained on large datasets and fine-tuned to understand the context and semantics of sentences. The specific model we will use is *"all-MiniLM-L6-v2"*, a lightweight and efficient transformer model. This model produces output vectors of size 384, providing a compact yet powerful representation of the tweets.

In [11]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using this sentence transformer, let's create embeddings for all of the patient case reports in the data set and add them to the Pandas DataFrame we created earlier. This may take a few moments.

In [12]:

# Generate embeddings for all tweets at once. Batch processing makes it faster
embeddings = model.encode(df_cases['note'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df_cases['note_vector'] = embeddings.tolist()


Let's explore the *df_cases* dataframe again, this time using just the *head* call to see the first 100 entries in the data set. Notice the addition of the vector embeddings in the newly added *note_vector* column.

In [13]:
pd.set_option('display.max_colwidth', 200)
df_cases.head(100)

,note,note_vector
0,"A a sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintai...","[-0.003768673399463296, 0.03895515576004982, -0.05829150229692459, 0.04102860763669014, -0.06328313052654266, 0.001131447614170611, -0.016128946095705032, 0.16160601377487183, -0.03382598981261253..."
1,This is the case of a 56-year-old man that was complaining of a dump pain on the right back and a swelling right in this place for several weeks. The patient was in good state and very active. The...,"[0.0572340302169323, 0.041176632046699524, 0.0032183341681957245, -0.007681556046009064, 0.007390230894088745, -0.03798162564635277, -0.023511353880167007, 0.07288163900375366, 0.01691499352455139..."
2,A 36-year old female patient visited our hospital with a chief complaint of pain and restricted range of motion (ROM) in the left hip joint persisting for two months. Physical examination of the p...,"[-0.0623912513256073, 0.04118657112121582, 0.0014734386932104826, 0.01914328895509243, -0.0012152084382250905, -0.045555565506219864, -0.0201222263276577, 0.07721439749002457, -0.00888017751276493..."
3,A 49-year-old male presented with a complaint of pain in the left proximal forearm after a fall. The patient had a history of left elbow arthrodesis performed for posttraumatic arthritis at the ag...,"[0.007449082098901272, -0.02825922518968582, -0.0004280074208509177, 0.012691575102508068, -0.02880946733057499, -0.03780525550246239, 0.02653159759938717, 0.05620591342449188, 0.01349935401231050..."
4,"A 47-year-old male patient was referred to the rheumatology clinic because of recurrent attacks of pain in both knees over 1 year.\nIn September 2016, the patient presented with severe pain over t...","[-0.04132620245218277, 0.007342552300542593, 0.015028499066829681, 0.024694137275218964, 0.008660009130835533, -0.09895151853561401, -0.03629561513662338, 0.10917240381240845, 0.03400370478630066,..."
5,A 24-year-old Yemeni female presented to the endocrinology clinic in April 2017 because of inability to walk and a long history of osteomalacia. She was unable to walk for the last 4 years with se...,"[-0.024297717958688736, 0.00462634302675724, -0.010054165497422218, 0.09496288746595383, -0.049306634813547134, -0.016045182943344116, -0.050467599183321, 0.08594101667404175, 0.015044580213725567..."
6,We report a 24-day-old female baby who presented with dyspnea and fever. Examination showed decreased air entry on the left side and a subsequent chest radiograph showed what looked like a consoli...,"[0.010019288398325443, 0.0526592917740345, 0.03426669165492058, -0.02406088262796402, -0.0035405410453677177, -0.09283165633678436, -0.01653829589486122, 0.07399806380271912, 0.08262445032596588, ..."
7,A 16 years old female patient presented to us with inability to walk on both lower limbs since last 3 months. Earlier the patient was able to walk with a limp and would sway to either side while w...,"[0.015219295397400856, -0.030706683173775673, 0.00041260113357566297, 0.04024651274085045, -0.030633393675088882, -0.008491715416312218, -0.010272125713527203, 0.11397922784090042, 0.0133814038708..."
8,We present a case of a seventy-three-year-old Saudi man who has started visiting the primary health care center in our institution twenty-five years ago. He has been concerned with having a cancer...,"[0.034984488040208817, 0.09654078632593155, -0.04681168869137764, 0.0038412881549447775, -0.054949767887592316, -0.005713239777833223, 0.012074858881533146, 0.015140339732170105, 0.059081953018903..."
9,"A 23-year-old female patient was admitted to a plastic surgery clinic because of an esthetic problem caused by swelling on the scalp, which started 1 year previously and gradually worsened in the ...","[-0.017202917486429214, 0.0267545934766531, 0.059389643371105194, 0.018649041652679443, -0.06511366367340088, -0.06519388407468796, 0.013

Next, let's load this into InterSystems IRIS by inserting each case report and its associated vector from the Pandas DataFrame into the *case_reports_sql* table we created earlier.

In [14]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df_cases.iterrows():
            sql = text("""
                INSERT INTO case_reports_sql 
                (note, note_vector) 
                VALUES (:note, TO_VECTOR(:note_vector))
            """)
            conn.execute(sql, {
                'note': row['note'], 
                'note_vector': str(row['note_vector'])
            })


### Run a vector search
With case reports loaded into InterSystems IRIS and vector embeddings stored alongside each one, let's run a vector search!

The block below will take a search phrase -- in this case, "Knee pain, child" -- and convert it into a vector to be used in searching for similar content.

In [15]:
note_search = "Knee pain, child"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

Next, we will use the vector that was just created based on the "Knee pain, child" search phrase and find the top three vectors that are closest in similarity to that vector. In this case, we are using the dot product of the vectors to determine their similarity; other methods include cosine similarity or Euclidian distance.

Run the block below to return the three case reports that our vector search indicates are most similar to the "Knee pain, child" search phrase.

In [16]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM case_reports_sql
            ORDER BY VECTOR_DOT_PRODUCT(note_vector, TO_VECTOR(:search_vector, DOUBLE)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


To print a readable output, we can take the results and process them for display using the block below.

In [17]:
results_df = pd.DataFrame(results, columns=df_cases.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,note
0,"A 14-year-old healthy male child presented to accident & emergency department with complaints of inability to walk or bear weight on his left lower limb after he spontaneously dislocated his patella while running on uneven ground. Upon further inquiry, the patient gave a history of similar episode 10 months back following a fall. At that time, his mother pushed the patella back after which he was taken to the hospital where radiographic images showed that patella was reduced with no associated fracture of patella, tibial tuberosity or femoral condyles. He was given a full length leg cast for 1 month after which he was able to resume his daily activities without any difficulty. There was a family history of recurrent patellar dislocation (father). Previous hospital records revealed that he had history of anterior knee pain with a positive apprehension sign indicating lateral patellar instability.\nOn examination, the knee was markedly swollen with obvious deformity in the lateral aspect in the form of tenting of the skin and soft tissue by underlying patella without any contusion or bruise. The knee was locked in 15 degrees of flexion and patient was unable to perform any movements at the knee joint. A provisional diagnosis of lateral dislocation of patella was made. Standard anteroposterior and lateral plain radiographs were taken which showed a laterally displaced and vertically rotated patella along its long axis with the medial patellar edge locked and dipping into the lateral gutter (). There was no evidence of associated fracture or signs of osteochondral damage. General examination showed no hyperlaxity of the joints. Clinical signs of genu valgum, patella alta, tibial torsion or trochlear dysplasia were absent and the Quadriceps (Q) angle on the normal limb was within normal range (11 degrees).\nAn attempt to closed reduction was made under conscious sedation but was not successful. A decision for open reduction was made and patient was shifted to operating room. A midline vertical incision was given along the knee joint. The patella was found to be locked with its"
1,"An 11-year-old boy was transferred to our institution with right knee pain and swelling after a fall while skiing occurred on the same day. He described the hitting of the frontal aspect of his leg. A physical examination revealed swelling and tenderness on the patella, the lateral aspect of the distal femur, and the medial aspect of the proximal tibia. Because of the pain, the knee could not be examined properly. The range of motion was severely limited by the pain, but a neurovascular examination was normal. Standard radiographs showed prepatellar intra-articular effusion and an isolated avulsion fracture with the elevation of the tibial attachment of the PCL (\n). The diagnosis was subsequently confirmed by computed tomographic scanning, and other bone lesions were excluded (\n). The tibial fragment measured 11 × 4 mm and presented a maximal displacement of 7 mm. No other ligamentous, meniscal, or chondral injuries were observed in a magnetic resonance imaging examination (\n). Because of minimal displacement, we decided to treat the avulsion fracture in a conservative way. The knee was immobilized for 6 weeks, with a long leg cast with 30 degrees of knee flexion. The patient was asked to walk with crutches, avoiding weight bearing. After removing the cast, the patient was allowed to begin gentle range-of-motion activities and weight bearing. The patient was asked to report for regular clinical and radiological controls every 4 to 6 weeks until 3 months after the trauma. No pain or instability was detected during a physical examination, and magnetic resonance imaging showed progressive consolidation of the fracture over time. Subsequently, the patient was allowed to progressively return to sport activities, reporting only rare episodes of knee joint swelling and slight pain during severe exertion. In addition, the patient was asked to fill in 